# Greatest common divisor (gcd)

The problem is to calculate gcd using integer programming. This is a direct formulation proposed by ChatGPT (as of 12/29/2024) but it is non-linear and will not work with linear solvers. ChatGPT provided a solution using GLPK linear solver.

Maximize $( Z = gcd )$

Subject to:
$$
\begin{aligned}
& A, B, gcd \in \mathbb{Z} \\
& A, B, gcd > 0 \\
& gcd \in [1, min(a, b)] \\
& gcd \cdot A = a \\
& gcd \cdot B = b \\
\end{aligned}
$$

Problem is taken from Mustafa Ç. Pınar's and  Deniz Akkaya's [Problems and Solutions for Integer and Combinatorial Optimization: Building Skills in Discrete Optimization](https://epubs.siam.org/doi/book/10.1137/1.9781611977769).

Assumptions: input values $a, b \in \mathbb{Z}, a > 0, b > 0$

In [4]:
import time
import pyomo.environ as pyo
from hypothesis import given, settings, strategies as st

#%matplotlib qt
%matplotlib inline

## Standard implementation and tests

Standard implementation uses Euclidean algorithm and property-based test.

In [5]:
def gcd(a, b, *args, **kwargs):
    """Compute the greatest common divisor of a and b using Euclidean algorithm (standard implementation)."""
    
    while b:
        a, b = b, a % b
    return {'gcd':a}

gcd(13, 19)

{'gcd': 1}

In [6]:
# Property-based testing - using hypothesis library
@settings(max_examples=50, deadline=None)
@given(a=st.integers(min_value=1, max_value=100), b=st.integers(min_value=1, max_value=100))
def test_gcd_function(a, b, gcd_function, solver):
    gcd_standard = gcd(a, b)['gcd']
    gcd_mip_result = gcd_function(a, b, solver=solver)
    gcd_mip = gcd_mip_result['gcd']
    assert gcd_standard == gcd_mip, \
        f"Failed for gcd_ip0 with a={a}, b={b}, gcd_ip={gcd_mip}, termination message: {gcd_mip_result['termination_message']}"

In [7]:
test_gcd_function(gcd_function=gcd, solver='cbc')

## GCD formulation using equality constraint

Minimize $( Z = A \cdot b + B \cdot a )$

Subject to:
$$
\begin{aligned}
& A, B \in \mathbb{Z} \\
& A \in [1, a] \\
& B \in [1, b] \\
& a \cdot B = A \cdot b \\
\end{aligned}
$$

Solution is unique:
$$
\begin{aligned}
& gcd = \frac{a}{A} = \frac{b}{B} \\
\end{aligned}
$$

In [8]:
def gcd_ip0(a, b, solver='cbc', filename=None):
    """Compute the greatest common divisor of a and b using MIP
    
    Uses the following model formulation (using equality constraint):
    min A*b + B*a
    s.t. A*b = B*a
    A, B >= 1, A <= a, B <= b

    gcd(a, b) = a/A = b/B
    """
    # Initialize the model
    model = pyo.ConcreteModel()

    # Define decision variables
    model.A = pyo.Var(domain=pyo.PositiveIntegers, bounds=(1, a))
    model.B = pyo.Var(domain=pyo.PositiveIntegers, bounds=(1, b))

    # Define the objective
    model.objective = pyo.Objective(
        expr=model.A*b + model.B*a, sense=pyo.minimize
    )

    # Define constraint
    model.constraint = pyo.Constraint(expr= model.A*b == model.B*a)
    
    # Write the model to a file
    if filename:
        model.write(filename, format='lp')

    # Solve the model and time it
    solver = pyo.SolverFactory(solver)
    start_time = time.time()
    result = solver.solve(model)
    elapsed_time = time.time() - start_time

    # Compile the results into a dictionary and return
    results = {
        'a': a,
        'b': b,
        'gcd': pyo.value(a/model.A),
        'time': elapsed_time,
        'Status': result.solver.status,
        'Termination Condition': result.solver.termination_condition,
        'Termination Message': result.solver.termination_message,
        'Optimal Value (z)': model.objective()
    }
    return results

In [9]:
gcd_ip0(9, 27, solver='cbc')

{'a': 9,
 'b': 27,
 'gcd': 9.0,
 'time': 0.15268802642822266,
 'Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.',
 'Optimal Value (z)': 54.0}

In [10]:
test_gcd_function(gcd_function=gcd_ip0, solver='cbc')

## GCD formulation using non-equality constraints

Minimize $( Z = A \cdot a + B \cdot b )$

Subject to:
$$
\begin{aligned}
& A, B \in \mathbb{Z} \\
& Z \geq 1 \\
\end{aligned}
$$

There are infinitely many solutions when A and B are not constrained:
$$
\begin{aligned}
& gcd = Z \\
\end{aligned}
$$

CBC wonders off to very large values of A and B which reveals a bug in displaying very large integers. See https://github.com/coin-or/Cbc/issues/685 for more details.


In [11]:
def gcd_ip1(a, b, solver='cbc'):
    """Compute the greatest common divisor of a and b using MIP (alternative formulation)
    
    Uses the following model (alternative formulation using inequality constraint):
    min A*a + B*b
    s.t. A*a + B*b >= 1

    gcd(a, b) = A*a + B*b    
    """
    # Initialize the model
    model = pyo.ConcreteModel()

    # Define decision variables
    model.A = pyo.Var(domain=pyo.Integers)
    model.B = pyo.Var(domain=pyo.Integers)
    model.C = pyo.Var(domain=pyo.PositiveIntegers)

    # Define the objective
    model.objective = pyo.Objective(
        expr=model.A*a + model.B*b, sense=pyo.minimize
    )

    # Define constraint
    model.constraint1 = pyo.Constraint(expr= (a*model.A + b*model.B) == model.C)
    
    
    # Solve the model and time it
    solver = pyo.SolverFactory(solver)
    start_time = time.time()
    result = solver.solve(model)
    elapsed_time = time.time() - start_time

    # Compile the results into a dictionary and return
    results = {
        'a': a,
        'b': b,
        'A': pyo.value(model.A),
        'B': pyo.value(model.B),
        'gcd': pyo.value(model.A*a + model.B*b),
        'time': elapsed_time,
        'Status': result.solver.status,
        'Termination Condition': result.solver.termination_condition,
        'Termination Message': result.solver.termination_message,
        'Optimal Value (z)': model.objective()
    }
    return results

In [12]:
test_gcd_function(gcd_function=gcd_ip1, solver='cbc')

KeyError: 'termination_message'

In [13]:
gcd_ip1(2, 2, solver='cbc')

{'a': 2,
 'b': 2,
 'A': -12345679000.0,
 'B': 12345679000.0,
 'gcd': 0.0,
 'time': 0.13756656646728516,
 'Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.',
 'Optimal Value (z)': 0.0}

## GCD formulation with non-equality constraints


Model formulation (c) using inequality constraints. Additional constraints ensure unique solution:

Minimize $( Z = A \cdot a + B \cdot b )$

Subject to:
$$
\begin{aligned}
& A, B \in \mathbb{Z} \\
& A \in [-b, b] \\
& B \in [-a, a] \\
& Z \geq 1 \\
\end{aligned}
$$

Additional constraints on A and B ensure uniquess of the solution:
$$
\begin{aligned}
& gcd = Z \\
\end{aligned}
$$

In [14]:
def gcd_ip2(a, b, solver='cbc', filename=None):
    """Compute the greatest common divisor of a and b using MIP (alternative formulation)
    
    Uses the following model (alternative formulation using inequality constraint):
    min A*a + B*b
    s.t. A*a + B*b >= 1

    gcd(a, b) = A*a + B*b    
    """
    # Initialize the model
    model = pyo.ConcreteModel()

    # Define decision variables
    model.A = pyo.Var(domain=pyo.Integers, bounds=(-b, b))
    model.B = pyo.Var(domain=pyo.Integers, bounds=(-a, a))
        
    # Define the objective
    model.objective = pyo.Objective(
        expr=model.A*a + model.B*b, sense=pyo.minimize
    )

    # Define constraint
    model.constraint = pyo.Constraint(expr= (1, model.A*a + model.B*b, min(a, b)))

    # Write the model to a file
    if filename:
        model.write(filename, format='lp')
    
    # Solve the model and time it
    solver = pyo.SolverFactory(solver)
    start_time = time.time()
    result = solver.solve(model)
    elapsed_time = time.time() - start_time

    # Compile the results into a dictionary and return
    results = {
        'a': a,
        'b': b,
        'A': pyo.value(model.A),
        'B': pyo.value(model.B),
        'gcd': pyo.value(model.A*a + model.B*b),
        'time': elapsed_time,
        'Status': result.solver.status,
        'Termination Condition': result.solver.termination_condition,
        'Termination Message': result.solver.termination_message,
        'Optimal Value (z)': model.objective()
    }
    return results

In [15]:
gcd_ip2(9, 27, solver='cbc')

{'a': 9,
 'b': 27,
 'A': -26.0,
 'B': 9.0,
 'gcd': 9.0,
 'time': 0.132279634475708,
 'Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.',
 'Optimal Value (z)': 9.0}

In [16]:
test_gcd_function(gcd_function=gcd_ip2, solver='cbc')